In [2]:
def hashf(s):
    digest = hashes.Hash(hashes.SHA256(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

In [6]:
from cryptography.hazmat.primitives.asymmetric import dh,dsa
from cryptography.hazmat.backends import default_backend


#Generate some parameters for Diffie-Hellman
parameters_dh = dh.generate_parameters(generator=2,key_size=3072,
                                      backend=default_backend())

#Generate some parameters for DSA
parameters_dsa = dsa.generate_parameters(key_size=3072, 
                                         backend=default_backend())

def DHDSA(conn):
    """
        DH
   Generate Keys
    """
    #generate private key DH
    pk_DH = parameters_dh.generate_private_key()
    #gemerate public key DH
    pub_DH = pk_DH.public_key().public_bytes(encoding=serialization.Encoding.PEM,
                                             format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    
    """
        DSA
   Generate Keys 
    """
    
    #generate private key DSA
    pk_DSA = parameters_dsa.generate_private_key()
    #generate public key DSA
    pub_DSA = pk_DSA.public_key().public_bytes(encoding=serialization.Encoding.PEM,
                                               format=serialization.PublicFormat.SubjectPublicKeyInfo)

    """
    Share Key
    """
    
    #send public key DSA
    conn.send(pub_DSA)
    
    #signing 
    signature = pk_DSA.sign(pub_DH,hashes.SHA256())
    
    peer_pub_DSA = serialization.load_pem_public_key(conn.recv(),
                                                     backend=default_backend())
    
    #send public key DH and signature
    conn.send(pub_DH)
    conn.send(signature)
    
    
    #check if signature is ok
    try:
        peer_pub = conn.recv()
        sig = conn.recv()
        peer_pub_dsa.verify(sig,peer_pub,hashes.SHA256())
        print("ok dsa")
    except InvalidSignature:
        print("fail dsa")
        

    #shared_key calculation
    peer_pub_key = serialization.load_pem_public_key(peer_pub,
                                                    backend = default_backend())
    
    shared_key = pk_DH.exchange(peer_pub_key)
    
    #confirmation
    my_tag = hashf(bytes(shared_key))
    conn.send(my_tag)
    peer_tag = conn.recv()
    if my_tag == peer_tag:
        print('OK DH')
        return my_tag
    else:
        print('FAIL DH')
        
    #erase data
    pk_DH = None
    pub_DH = None
    peer_pub = None
    peer_pub_key = None
    shared_key = None
    my_tag = None
    peer_tag = None  

SyntaxError: EOL while scanning string literal (<ipython-input-6-5804a32f8d31>, line 28)

In [ ]:
from multiprocessing import Pipe, Process

class Connector(object):
    def __init__(self,left,right,timeout=None):
        """
        left : function that connects to the left of the Pipe
        right : function that connects to the right of the Pipe
        timeout : (optional) seconds before killing process
        """
        #Pipe function returns a pair of connection objects 
        #connected by a Pipe (default two-way)
        left_end, right_end = Pipe()
        self.timeout = timeout
    
        # processes connected to Pipe
        self.lproc = Process(target=left, args=(left_end,))
        self.rproc = Process(target=right, args=(right_end,))
        
        #functions connected to Pipe
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join()
    

In [ ]:
def Emitter(conn):
    conn.send('bitch')
    conn.close()

In [ ]:
def Receiver(conn):
    i=0
    while i!=5:
        stra = conn.recv()
        i = len(stra)
        print(stra)

In [ ]:
Connector(Emitter,Receiver,timeout=30).auto()